# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit import Particle

In [2]:
t0 = Time("2024-12-01 00:00")
t1 = Time("2025-12-01 00:00")

obj = Horizons(id="274301", location="@0", epochs=[t0.tdb.jd, t1.tdb.jd])
vecs = obj.vectors(refplane="earth")

x0 = jnp.array([vecs["x"][0], vecs["y"][0], vecs["z"][0]])
v0 = jnp.array([vecs["vx"][0], vecs["vy"][0], vecs["vz"][0]])

In [3]:
p = Particle(x=x0, v=v0, log_gm=-jnp.inf, time=t1, gravity="newtonian planets")
p

Particle: unnamed

In [4]:
p.integrate(t1)

(Array([[-1.78355289,  1.97281979,  0.57967129]], dtype=float64),
 Array([[-0.00752441, -0.00576865, -0.00178666]], dtype=float64))

In [9]:
from jorbit.integrators.ias15 import ias15_evolve_dynamic_steps, ias15_evolve

ias15_evolve(
    p.cartesian_state.to_system(),
    p.gravity,
    jnp.array([t1.tdb.jd]),
    p.integrator_state,
    10,
)

(Array([[[-1.78355289,  1.97281979,  0.57967129]]], dtype=float64),
 Array([[[-0.00752441, -0.00576865, -0.00178666]]], dtype=float64),
 SystemState(tracer_positions=Array([[-1.78355289,  1.97281979,  0.57967129]], dtype=float64), tracer_velocities=Array([[-0.00752441, -0.00576865, -0.00178666]], dtype=float64), massive_positions=Array([], shape=(0, 3), dtype=float64), massive_velocities=Array([], shape=(0, 3), dtype=float64), log_gms=Array([], shape=(0,), dtype=float64), time=Array(2461010.50080073, dtype=float64), acceleration_func_kwargs=None),
 <jorbit.integrators.IAS15IntegratorState at 0x134a54d60>)

In [12]:
from jorbit.integrators.ias15 import ias15_evolve_dynamic_steps, ias15_evolve

ias15_evolve(
    p.cartesian_state.to_system(),
    p.gravity,
    jnp.array([t1.tdb.jd]),
    p.integrator_state,
    1000,
)

(Array([[[-1.78355289,  1.97281979,  0.57967129]]], dtype=float64),
 Array([[[-0.00752441, -0.00576865, -0.00178666]]], dtype=float64),
 SystemState(tracer_positions=Array([[-1.78355289,  1.97281979,  0.57967129]], dtype=float64), tracer_velocities=Array([[-0.00752441, -0.00576865, -0.00178666]], dtype=float64), massive_positions=Array([], shape=(0, 3), dtype=float64), massive_velocities=Array([], shape=(0, 3), dtype=float64), log_gms=Array([], shape=(0,), dtype=float64), time=Array(2461010.50080073, dtype=float64), acceleration_func_kwargs=None),
 <jorbit.integrators.IAS15IntegratorState at 0x134a55700>)

In [10]:
ias15_evolve_dynamic_steps(
    p.cartesian_state.to_system(),
    p.gravity,
    jnp.array([t1.tdb.jd]),
    p.integrator_state,
)

(Array([[[-1.78355289,  1.97281979,  0.57967129]]], dtype=float64),
 Array([[[-0.00752441, -0.00576865, -0.00178666]]], dtype=float64),
 SystemState(tracer_positions=Array([[-1.78355289,  1.97281979,  0.57967129]], dtype=float64), tracer_velocities=Array([[-0.00752441, -0.00576865, -0.00178666]], dtype=float64), massive_positions=Array([], shape=(0, 3), dtype=float64), massive_velocities=Array([], shape=(0, 3), dtype=float64), log_gms=Array([], shape=(0,), dtype=float64), time=Array(2461010.50080073, dtype=float64), acceleration_func_kwargs=None),
 <jorbit.integrators.IAS15IntegratorState at 0x134a55e90>)

In [11]:
%%timeit

ias15_evolve(
    p.cartesian_state.to_system(),
    p.gravity,
    jnp.array([t1.tdb.jd]),
    p.integrator_state,
    10,
)[0].block_until_ready()

474 μs ± 15 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
%%timeit

ias15_evolve(
    p.cartesian_state.to_system(),
    p.gravity,
    jnp.array([t1.tdb.jd]),
    p.integrator_state,
    1000,
)[0].block_until_ready()

1.04 ms ± 17.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
%%timeit

ias15_evolve_dynamic_steps(
    p.cartesian_state.to_system(),
    p.gravity,
    jnp.array([t1.tdb.jd]),
    p.integrator_state,
)[0].block_until_ready()

521 μs ± 10.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
